In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
import os
import sys

import rosbag
import rospy



import sensor_msgs.point_cloud2 as pc2

In [ ]:
MAP_PATH = "/home/gonz/Desktop/THESIS/code/global-planning/gnd_dataset/ele2/ele2.pgm"
SEM_MAP_PATH = "/home/gonz/Desktop/THESIS/code/global-planning/gnd_dataset/ele2/ele2.png"
# SEM_MAP2_PATH = "/home/tesistas/Desktop/GONZALO/gnd_test/canchafullseg.png"

RES = 0.05
MAP_WIDTH = 0.5
MAP_HEIGHT = 0.5

In [ ]:
occ_map = cv.imread(MAP_PATH, cv.IMREAD_GRAYSCALE)
seg_map = cv.cvtColor(cv.imread(SEM_MAP_PATH), cv.COLOR_BGR2RGB)
# seg_map2 = cv.cvtColor(cv.imread(SEM_MAP2_PATH), cv.COLOR_BGR2RGB)

fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].imshow(occ_map, cmap='gray')
ax[0].set_title("Occupancy Map")
ax[0].set_xlabel("X (m)")
ax[0].set_ylabel("Y (m)")
ax[1].imshow(seg_map)
ax[1].set_title("Semantic Map")
ax[1].set_xlabel("X (m)")
ax[1].set_ylabel("Y (m)")

print("Occupancy Map Size: ", occ_map.shape)
print("Semantic Map Size: ", seg_map.shape)

In [ ]:
seg_map_gray = cv.cvtColor(seg_map, cv.COLOR_BGR2GRAY)

seg_map_gray[seg_map_gray != 81] = 1
seg_map_gray[seg_map_gray == 81] = 0

plt.imshow(seg_map_gray)

cv.imwrite("/home/gonz/Desktop/THESIS/code/global-planning/fcfm_navigation_dataset/ros_map_utils/maps/electricagray.png", seg_map_gray*255)

In [ ]:
occ_map = cv.imread(MAP_PATH, cv.IMREAD_GRAYSCALE)
seg_map = cv.cvtColor(cv.imread(SEM_MAP_PATH), cv.COLOR_BGR2RGB)
# seg_map2 = cv.cvtColor(cv.imread(SEM_MAP2_PATH), cv.COLOR_BGR2RGB)

fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].imshow(occ_map, cmap='gray')
ax[0].set_title("Occupancy Map")
ax[0].set_xlabel("X (m)")
ax[0].set_ylabel("Y (m)")
ax[1].imshow(seg_map)
ax[1].set_title("Semantic Map")
ax[1].set_xlabel("X (m)")
ax[1].set_ylabel("Y (m)")

print("Occupancy Map Size: ", occ_map.shape)
print("Semantic Map Size: ", seg_map.shape)

In [ ]:
# read a rosbag file

BAG_FILE = "/home/gonz/Desktop/bags/sec_a_3_2025-05-23-21-48-28.bag"

bag = rosbag.Bag(BAG_FILE, 'r')
bag.get_start_time()

odometry_xy = []
map_info = None
for topic, msg, t in bag.read_messages(topics=['/amcl_pose', "/map"]):
    # clouds.append(np.array(list(pc2.read_points(msg, field_names=("x", "y", "z"), skip_nans=True))))
    if topic == "/map":
        map_info = msg.info
        # occ_map = np.array(msg.data).reshape((msg.info.height, msg.info.width))
    else:
        odometry_xy.append([msg.pose.pose.position.x, msg.pose.pose.position.y])

bag.close()

odometry_xy = np.array(odometry_xy)
print("Odometry XY: ", odometry_xy.shape)
print("Map Info: ", map_info)

In [ ]:
def world_to_map(x, y):
    """
    Convert world coordinates to map coordinates.
    :param x: x coordinate in world
    :param y: y coordinate in world
    :return: x, y coordinates in map
    """
    x_map = (x - map_info.origin.position.x) / map_info.resolution
    y_map = (y - map_info.origin.position.y) / map_info.resolution
    return x_map, y_map

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].imshow(np.flipud(occ_map), cmap='gray')
ax[0].set_title("Occupancy Map")
ax[0].set_xlabel("X (m)")
ax[0].set_ylabel("Y (m)")
ax[1].imshow(np.flipud(seg_map))
ax[1].set_title("Semantic Map")
ax[1].set_xlabel("X (m)")
ax[1].set_ylabel("Y (m)")

xmap, ymap = world_to_map(odometry_xy[:, 0], odometry_xy[:, 1])
ax[1].plot(xmap, ymap, 'r-', label='Odometry Path')
ax[0].plot(xmap, ymap, 'r-', label='Odometry Path')

ax[1].scatter(xmap[-1], ymap[-1])

print("Occupancy Map Size: ", occ_map.shape)
print("Semantic Map Size: ", seg_map.shape)

In [ ]:
def get_map_slice(map, pose, size_m=15):
    px, py = world_to_map(pose[0], pose[1])
    px, py = int(px), int(py)
    size_px2 = int(size_m / RES)

    mapc = np.flipud(map).copy()
    map_slice = mapc[py-size_px2:py+size_px2, px-size_px2:px+size_px2]
    
    origin = [px, py]

    return map_slice, origin

In [ ]:
a=list(range(10))
print(a, a[:-1])

In [ ]:
def get_path_lenght_interval(odometry_xy, start, lenght=15., N_wpts=15):
    xsq = (odometry_xy[start+1:, 0] - odometry_xy[start:-1, 0]) ** 2
    ysq = (odometry_xy[start+1:, 1] - odometry_xy[start:-1, 1]) ** 2
    distances = np.cumsum(np.sqrt(xsq + ysq), axis=0)

    stop_idx = np.where(distances >= lenght)[0][0] + start
    print(distances[stop_idx-start])
    ids = np.linspace(start, stop_idx, N_wpts, dtype=np.int64) # inlcude last point

    return odometry_xy[ids]

In [ ]:
pt = 200

map_slice, ori = get_map_slice(occ_map, odometry_xy[pt])
map_slice_seg, _ = get_map_slice(seg_map, odometry_xy[pt])

print(odometry_xy[pt])
print(ori)

fig, ax = plt.subplots(1, 2, figsize=(10, 5))

px, py = world_to_map(odometry_xy[pt][0], odometry_xy[pt][1])
# px, py = 1250, 1500
# xmap_slice = xmap - px + int(15 / (RES * 2))
# ymap_slice = ymap - py + int(15 / (RES * 2))

# ax[1].plot(xmap_slice, ymap_slice, 'b-', label='Odometry Path')
# ax[0].plot(xmap_slice, ymap_slice, 'b-', label='Odometry Path')

map_drawn = map_slice_seg.copy()

sampled_path = get_path_lenght_interval(odometry_xy, pt)
for (x, y) in sampled_path:
    pxi, pyi = world_to_map(x, y)
    pxi = int(pxi - ori[0] + int(15 / (RES)))
    pyi = int(pyi - ori[1] + int(15 / (RES)))

    map_drawn = cv.circle(map_drawn, (pxi, pyi), 3, (255, 255, 255), -1)



ax[0].imshow(seg_map);
ax[1].imshow(map_drawn)
ax[0].axis('off')
ax[1].axis('off')


## Data organization.

What values do we need in one sample of data?

1. Local map 
2. past 10 values of velocity
3. past 3 values of pointcloud
4. relative Goal (is it relative? nice it actually is)
5. K GT trajectories.


### 1. Local map

In [ ]:
def rotate_image(image, angle, center=None):
  if center is None:
    center = tuple(np.array(image.shape[1::-1]) / 2)
  
  rot_mat = cv.getRotationMatrix2D(center, angle, 1.0)
  result = cv.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv.INTER_LINEAR)
  return result, rot_mat


def get_local_map(map, pose, size_m=15, flip=True, color=None):
    px, py = world_to_map(pose[0], pose[1])
    px, py = int(px), int(py)
    size_px2 = int(size_m / RES)

    if flip:
        mapc = np.flipud(map).copy()
    else:
        mapc = map.copy()

    mapc, R = rotate_image(mapc, pose[2] * 180 / np.pi, center=(px, py))

    map_slice = mapc[py-size_px2:py+size_px2, px-size_px2:px+size_px2]
    
    if color is not None:
        map_slice = cv.cvtColor(map_slice, cv.COLOR_BGR2GRAY)
        map_slice[map_slice != color] = 1
        map_slice[map_slice == color] = 0

    origin = [px, py]

    return map_slice, origin, R

### 2. Velocities

In [ ]:
from collections import deque
import random
from cv_bridge import CvBridge

odom_topic = "/panther/odometry/filtered"
scan_topic = "/repub/ouster/points"
amcl_topic = "/amcl_pose"
img_topic  = "/repub/camera/image_raw"

N_VEL = 10  # Number of odom messages
N_LIDAR = 3  # Number of lidar messages

# Load messages from the bag
odom_msgs = []
scan_msgs = []
amcl_msgs = []
img_msgs  = []

with rosbag.Bag(BAG_FILE, 'r') as bag:
    for topic, msg, t in bag.read_messages(topics=[odom_topic, scan_topic, amcl_topic, img_topic]):
        timestamp = t.to_sec()
        if topic == odom_topic:
            odom_msgs.append((timestamp, msg))
        elif topic == scan_topic:
            scan_msgs.append((timestamp, msg))
        elif topic == amcl_topic:
            amcl_msgs.append((timestamp, msg))
        elif topic == img_topic:
            img_msgs.append((timestamp, msg))

In [ ]:
# Select a random AMCL pose
amcl_msgs_lst = np.array([[msg.pose.pose.position.x, msg.pose.pose.position.y] for (t, msg) in amcl_msgs])

ind = np.random.choice(list(range(len(amcl_msgs))))
random_amcl = amcl_msgs[ind]

amcl_time, amcl_msg = random_amcl

# Helper to get last N messages before a given timestamp
def get_last_msgs(msgs, current_time, N):
    return [m for t, m in reversed(msgs) if t <= current_time][:N]

# Get previous K odom and N scan messages
odom_history = get_last_msgs(odom_msgs, amcl_time, N_VEL)
scan_history = get_last_msgs(scan_msgs, amcl_time, N_LIDAR)
img_history  = get_last_msgs(img_msgs, amcl_time, 1)

In [ ]:
def world_to_local(points_world, robot_pose):
    """
    Convert world-frame points to robot-centric local frame.

    :param points_world: Nx2 array of (x, y) points in meters
    :param robot_pose: tuple (x_r, y_r, theta_r)
    :return: Nx2 array of transformed points in local frame
    """
    x_r, y_r, theta_r = robot_pose

    # Translate points so robot is at the origin
    translated = points_world - np.array([x_r, y_r])

    # Rotation matrix to align robot's heading with +Y
    c, s = np.cos(-theta_r), np.sin(-theta_r)
    R = np.array([[c, -s],
                  [s,  c]])

    local_points = translated @ R.T
    return local_points

In [ ]:
from tf.transformations import euler_from_quaternion
bridg = CvBridge()
img = bridg.imgmsg_to_cv2(img_history[0])


q = amcl_msg.pose.pose.orientation
_, _, yaw = euler_from_quaternion([q.x, q.y, q.z, q.w])

pose = [amcl_msg.pose.pose.position.x, amcl_msg.pose.pose.position.y, yaw + np.pi / 2]  # Convert to radians and adjust orientation

local_map, ori, R = get_local_map(seg_map, pose, color=81)

print("Local Map Origin: ", ori)
print("Rotation Matrix:\n ", R)


sampled_path = get_path_lenght_interval(amcl_msgs_lst, ind)
local_path = world_to_local(sampled_path, pose)

a = local_map.copy()
for (x, y) in local_path:
    pxi = int(x / RES + int(15 / RES))
    pyi = int(y / RES + int(15 / RES))
    a = cv.circle(a, (pxi, pyi), 2, (1), -1)

plt.imshow(img)
plt.imshow(np.flipud(a))

### 3. Lidar

In [ ]:
# done above i think
pcl = np.array(list(pc2.read_points(scan_history[0], field_names=("x", "y", "z", "intensity"), skip_nans=True)))

# 3d plot of the point cloud
ax = plt.figure().add_subplot(projection='3d')
ax.scatter3D(pcl[:, 0], pcl[:, 1], pcl[:, 2], cmap='Greens')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_title('3D Point Cloud')
plt.show()

### 4. Goal

### 5. GT trajectories

In [ ]:
import pickle
with open('/home/gonz/Desktop/THESIS/code/global-planning/gnd_dataset/local_map_files_120/paths/108_0.pkl', 'rb') as f:
    data = pickle.load(f)

print(data.keys())
len(data['lidar_array'])

plt.imshow(data['camera'][0])
plt.imshow(data['local_map'])
plt.plot(data['path'][:, 1]/0.1+300, data['path'][:, 0]/0.1+300, 'r-')


print(data['all_paths'][0].shape)

plt.scatter(data['lidar'][0][:, 1]/0.1+300, data['lidar'][0][:, 0]/0.1+300, s=1, c='b', label='Scan Points')
plt.ylim(600, 0)
plt.xlim(0, 600)

leng = 0
for i in range(len(data['path']) - 1):
    leng += np.linalg.norm(data['path'][i] - data['path'][i + 1])
print(leng)

In [ ]:
path = data['path']
# path[:, ::-1]
path

In [ ]:
def rotate_pointcloud(pcl, theta_rad):
    # Rotation matrix around Z
    R = np.array([
        [np.cos(theta_rad), -np.sin(theta_rad), 0],
        [np.sin(theta_rad),  np.cos(theta_rad), 0],
        [0,                  0,                 1]
    ])
    return pcl @ R.T  # Rotate each point

In [ ]:
import pickle
with open('/home/gonz/Desktop/THESIS/code/global-planning/gnd_dataset/local_map_files_120/paths/1150_0.pkl', 'rb') as f:
    data = pickle.load(f)

print(data.keys())

plt.imshow(data['camera'][0])
plt.imshow(data['local_map'])
plt.plot(data['path'][:, 1]/0.1+300, data['path'][:, 0]/0.1+300, 'r-')

print(data['lidar'][0].shape)
pcl = data['lidar'][0]

plt.scatter(pcl[:, 1]/0.1+300, pcl[:, 0]/0.1+300, s=1, c='b', label='Scan Points')
plt.ylim(600, 0)
plt.xlim(0, 600)

leng = 0
for i in range(len(data['path']) - 1):
    leng += np.linalg.norm(data['path'][i] - data['path'][i + 1])
print(data['pose'])

In [ ]:
import pickle

n = 1087
with open(f'/home/gonz/Desktop/THESIS/code/global-planning/gnd_dataset/local_map_files_120/paths/{n}_0.pkl', 'rb') as f:
    data = pickle.load(f)

with open('/home/gonz/Desktop/THESIS/code/global-planning/gnd_dataset/local_map_files_120/paths/djisktra_paths.pkl', 'rb') as f:
    paths = pickle.load(f)

print(len(paths[f'{n}_0']))
print(data.keys())



plt.imshow(data['camera'][0])
plt.imshow(data['local_map'])
plt.plot(data['path'][:, 1]/0.1+300, data['path'][:, 0]/0.1+300, 'r-')

for path in paths[f'{n}_0']:
    path = world_to_local(path, data['pose'])
    plt.plot(path[:, 0]/0.1+300, path[:, 1]/0.1+300, 'g-')

print(data['lidar'][0].shape)
pcl = data['lidar'][0]

# plt.scatter(pcl[:, 1]/0.1+300, pcl[:, 0]/0.1+300, s=1, c='b', label='Scan Points')
# plt.ylim(600, 0)
# plt.xlim(0, 600)

leng = 0
for i in range(len(data['path']) - 1):
    leng += np.linalg.norm(data['path'][i] - data['path'][i + 1])
print(leng)

### que es lo que falta realmente:

1. generar todos los pkl sin gt (para todos los cosos).
2. generar todos los gt a partir de los pkl
3. mergear los datos. Nuevo modo de la funcion original para esto.
4. sfijarse bien en los x, y
5 entrenar mierda.
